# Svenskt Kvinnobiografiskt lexikon part 1
version 1.3.4.34


* Denna [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) 
  * [del 2](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%202.ipynb) med sökfrågor etc.
  * [del 3](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb) hämtar data
* Video om detta [svenska](https://www.youtube.com/watch?v=_LP48Yf6vgI) / [engelska](https://www.youtube.com/watch?v=wolyLu0WVsA)  
* Wikidata - [An introduction to Wikidata](https://www.youtube.com/watch?v=m_9_23jXPoE)
  * SKBL WIkidata property [P4963](https://www.wikidata.org/wiki/Property_talk:P4963) 
  

# Check diff SKBL Wikidata 
ett försök att hålla Wikidata / SKBL i fas 
* [KARP APIet](https://spraakbanken.gu.se/verktyg/karp/webb-api)
* se tidigare försök att ha ISNI i synk [T219700](https://phabricator.wikimedia.org/T219700), GITHUB [salgo60/SKBLWikidata](https://github.com/salgo60/SKBLWikidata) 
  * mar 2019 kommentar om [platser i SKBL](https://phabricator.wikimedia.org/T218782)  
  
Viss osäkerhet hur en bra API fråga skall se ut returnerar alla personer i SKBL

In [1]:
import urllib3, json
import pandas as pd 
from datetime import datetime
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', None)

now = datetime.now()
print("Last run: ", now)

dftot = pd.DataFrame()
 
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/minientry?q=extended%7C%7Cand%7Cnamn%7Cexists&mode=skbllinks&show=name%2Curl&sort=sorteringsnamn.sort%2Csorteringsnamn.init%2Ctilltalsnamn.sort&size=10000&resource=skbl"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())
listSKBL = []
i = 0 
for row in (data["hits"]["hits"]):
    #print (row["_source"]["url"])
    listSKBL.append(row["_source"]["url"])
    i = i + 1 
    #print (i)
SKBLtot = pd.DataFrame(listSKBL,
                  columns=['skblid'])


Last run:  2020-09-04 16:36:18.376547


In [2]:
SKBLtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544 entries, 0 to 1543
Data columns (total 1 columns):
skblid    1544 non-null object
dtypes: object(1)
memory usage: 12.2+ KB


In [3]:
#SKBLtot

In [4]:
#SKBLtot.sort_index(inplace=True) 
SKBLtot.sort_values(['skblid'], ascending=[1],inplace=True) 

In [5]:
# The API gives us some duplicates 
SKBLtot.drop_duplicates(subset ="skblid", keep = False, inplace = True)
SKBLtot.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415 entries, 550 to 831
Data columns (total 1 columns):
skblid    1415 non-null object
dtypes: object(1)
memory usage: 22.1+ KB


In [6]:
SKBLtot = SKBLtot.reset_index(drop=True)  
#SKBLtot

# Wikidata

In [7]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skbl
} order by ?skbl"""


def get_results(endpoint_url, query):
    user_agent = "User:salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)
 
lst = []
for result in results["results"]["bindings"]:
    lst.append((result["skbl"]["value"]))
WDtot = pd.DataFrame(lst,columns={'skblid'}) 
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1475 entries, 0 to 1474
Data columns (total 1 columns):
skblid    1475 non-null object
dtypes: object(1)
memory usage: 11.6+ KB


In [8]:
WDtot.sort_index(inplace=True)  

In [9]:
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1475 entries, 0 to 1474
Data columns (total 1 columns):
skblid    1475 non-null object
dtypes: object(1)
memory usage: 11.6+ KB


In [10]:
#WDtot

### Check duplicates SKBL and Wikidata

In [11]:
dfmerge = pd.merge(WDtot, SKBLtot,how='outer', indicator=True)

In [12]:
# check whatsnew / deleted  
dfmerge['_merge'].value_counts()

both          1412
left_only       63
right_only       3
Name: _merge, dtype: int64

In [13]:
#dfmerge

In [14]:
SKBLnew = dfmerge[dfmerge['_merge']=="right_only"]


In [15]:
SKBLnew

,skblid,_merge
1475,BarbroKerstinElisabetSoderberg,right_only
1476,ClaraNataliaTeresiaEuren,right_only
1477,MarianneLindsten,right_only


In [16]:
SKBLnew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 1475 to 1477
Data columns (total 2 columns):
skblid    3 non-null object
_merge    3 non-null category
dtypes: category(1), object(1)
memory usage: 155.0+ bytes


In [17]:
import csv  
jsonURL = "https://skbl.se/sv/artikel/"
listNewItems =[]
for index,row in SKBLnew.iterrows():
    url = jsonURL + row["skblid"] + ".json"
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))  
    
    new_item = dict()
    try:
        publishedDate = data['publication_date'],        
    except:
        publishedDate =""
    new_item['publishedDate'] = publishedDate[0]
    try:
        birthdate = data['lifespan']['from']['date']['date'],
    except:
        birtdate =""
    new_item['birthdate'] = birthdate[0]
    try:
        deathdate = data['lifespan']['to']['date']['date']
    except:
        deathdate=""
    new_item['deathdate'] = deathdate
    try:
        birthparish = data['lifespan']['from']['place']['parish']
    except:
        birthparish = ""        
    new_item['birthparish'] = birthparish
    try:
        deathparish = data['lifespan']['to']['place']['parish']
    except:
        deathparish = ""
    new_item['deathparish'] = deathparish
    try:
        ISNI = data['id']['id']
    except:
        ISNI = ""
    new_item['ISNI'] = ISNI
    new_item['SKBL'] = row["skblid"]
    new_item['index'] = index
    new_item['skbl_status'] = data['skbl_status']
    new_item['firstname'] = data['name'].get('firstname')
    new_item['lastname'] = data['name'].get('lastname')
    new_item['subtitle'] = data['subtitle']
    listNewItems.append(new_item)
print (len(listNewItems) ," antal poster")
if len(listNewItems) > 0:
    keys = listNewItems[0].keys()
    with open("SKBL_newfile.csv", "w", newline='') as SKBLfile:
        dict_writer = csv.DictWriter(SKBLfile, keys)
        dict_writer.writeheader()
        dict_writer.writerows(listNewItems)

    dfSKBLcsv = pd.read_csv("SKBL_newfile.csv", sep=",")   
    dfSKBLcsv
else:
    print ("Ingen fil skapas inga nya poster") 
            


3  antal poster


In [18]:
end = datetime.now()
print("Ended: ", end)

Ended:  2020-09-04 16:36:21.040456
